<a href="https://colab.research.google.com/github/yeamuhid/Meching-Learning-/blob/main/Reinforcement_of_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gym
import numpy as np
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load pre-trained model for transfer
def load_pretrained_model(filepath, new_output_dim):
    # Load the pre-trained model
    model = load_model(filepath)
    # Remove the last layer and add a new output layer
    model.pop()
    model.add(Dense(new_output_dim))


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [3]:
import gym
import numpy as np
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load pre-trained model and adapt it to new task
def create_model_from_pretrained(filepath, input_dim, output_dim):
    try:
        # Load the pre-trained model
        pretrained_model = load_model(filepath)
        # Remove the last layer (if dimensions don't match) and add a new output layer
        model = Sequential(pretrained_model.layers[:-1])
    except Exception as e:
        print("Pretrained model could not be loaded. Creating a new one.")
        model = Sequential()
        model.add(Dense(24, input_dim=input_dim, activation='relu'))
        model.add(Dense(24, activation='relu'))
    # Add output layer specific to the new task
    model.add(Dense(output_dim, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Simple Q-learning algorithm with transfer
def train_rl_model(env_name, pretrained_filepath=None, episodes=500):
    # Initialize the environment
    env = gym.make(env_name)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    # Load or create the RL model
    model = create_model_from_pretrained(pretrained_filepath, input_dim=state_size, output_dim=action_size)

    # Hyperparameters
    gamma = 0.95   # Discount factor
    epsilon = 1.0  # Exploration rate
    epsilon_decay = 0.995
    epsilon_min = 0.01
    batch_size = 32
    replay_buffer = []

    for episode in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        while True:
            # Exploration vs Exploitation
            if np.random.rand() <= epsilon:
                action = np.random.choice(action_size)
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values[0])

            # Take the action
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            total_reward += reward

            # Store in replay buffer
            replay_buffer.append((state, action, reward, next_state, done))
            if len(replay_buffer) > 2000:  # Limit buffer size
                replay_buffer.pop(0)

            # Train the model using samples from the replay buffer
            if len(replay_buffer) > batch_size:
                minibatch = np.random.choice(replay_buffer, batch_size)
                for s, a, r, s_next, d in minibatch:
                    target = r
                    if not d:
                        target += gamma * np.amax(model.predict(s_next)[0])
                    target_f = model.predict(s)
                    target_f[0][a] = target
                    model.fit(s, target_f, epochs=1, verbose=0)

            state = next_state

            if done:
                print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")
                break

        # Decay exploration rate
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

    # Save the trained model
    model.save("trained_rl_model.h5")
    return model

# Example usage
if __name__ == "__main__":
    trained_model = train_rl_model(env_name="CartPole-v1", pretrained_filepath="pretrained_model.h5")


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_check

Pretrained model could not be loaded. Creating a new one.
Episode 1/500, Total Reward: 19.0


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (33, 5) + inhomogeneous part.

In [ ]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 1. Create a base model (pre-trained model)
def create_base_model(input_dim, output_dim):
    model = Sequential([
        Dense(24, input_dim=input_dim, activation='relu'),
        Dense(24, activation='relu'),
        Dense(output_dim, activation='linear')  # Output layer for Q-values
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 2. Transfer learning: modify the model for the new task
def transfer_model(base_model, new_output_dim):
    base_model.pop()  # Remove the old output layer
    base_model.add(Dense(new_output_dim, activation='linear'))  # Add new output layer
    base_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return base_model

# 3. Define the RL environment
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Assume base model was trained for a different environment
# Create or load pre-trained base model
pretrained_model = create_base_model(input_dim=state_size, output_dim=2)  # Example base model

# Transfer the model for the new task
model = transfer_model(pretrained_model, new_output_dim=action_size)

# 4. Implement Q-Learning with the transferred model
def train_dqn(env, model, episodes=1000, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
    for episode in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        for time in range(500):  # Max time steps
            # Choose action (epsilon-greedy)
            if np.random.rand() <= epsilon:
                action = np.random.choice(action_size)
            else:
                q_values = model.predict(state, verbose=0)
                action = np.argmax(q_values[0])

            # Take action
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            # Update Q-values
            target = reward
            if not done:
                target += gamma * np.max(model.predict(next_state, verbose=0)[0])
            q_values = model.predict(state, verbose=0)
            q_values[0][action] = target

            # Train the model
            model.fit(state, q_values, verbose=0)

            state = next_state
            total_reward += reward

            if done:
                print(f"Episode {episode + 1}/{episodes}, Reward: {total_reward}")
                break

        # Reduce exploration rate
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

# 5. Train the model with transfer learning
train_dqn(env, model)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/ker

Episode 1/1000, Reward: 16.0
Episode 2/1000, Reward: 8.0
Episode 3/1000, Reward: 16.0
Episode 4/1000, Reward: 13.0
Episode 5/1000, Reward: 29.0
Episode 6/1000, Reward: 16.0
Episode 7/1000, Reward: 14.0
Episode 8/1000, Reward: 33.0
Episode 9/1000, Reward: 29.0
Episode 10/1000, Reward: 53.0
Episode 11/1000, Reward: 15.0
Episode 12/1000, Reward: 16.0
Episode 13/1000, Reward: 18.0
Episode 14/1000, Reward: 12.0
Episode 15/1000, Reward: 16.0
Episode 16/1000, Reward: 23.0
Episode 17/1000, Reward: 26.0
Episode 18/1000, Reward: 12.0
Episode 19/1000, Reward: 27.0
Episode 20/1000, Reward: 15.0
Episode 21/1000, Reward: 16.0
Episode 22/1000, Reward: 22.0
Episode 23/1000, Reward: 11.0
Episode 24/1000, Reward: 58.0
Episode 25/1000, Reward: 16.0
Episode 26/1000, Reward: 11.0
Episode 27/1000, Reward: 34.0
Episode 28/1000, Reward: 18.0
Episode 29/1000, Reward: 26.0
Episode 30/1000, Reward: 14.0
Episode 31/1000, Reward: 26.0
Episode 32/1000, Reward: 14.0
Episode 33/1000, Reward: 22.0


In [ ]:
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

# 1. Define the base model (pre-trained model)
def create_base_model(input_dim, output_dim):
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu'),
        Dense(128, activation='relu'),
        Dense(output_dim, activation='linear')  # Q-value output
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 2. Transfer learning: Adapt the model to the new task
def transfer_model(base_model, new_output_dim):
    base_model.pop()  # Remove the old output layer
    base_model.add(Dense(new_output_dim, activation='linear'))  # Add new output layer
    base_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return base_model

# 3. Define the Replay Buffer for experience storage
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def size(self):
        return len(self.buffer)

# 4. Deep Q-Learning with Transfer Learning
def train_dqn_with_transfer(env, base_model, episodes=500, gamma=0.99, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995,
                             batch_size=64, buffer_size=100000, update_target_freq=10):
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    # Transfer model
    model = transfer_model(base_model, new_output_dim=action_size)
    target_model = clone_model(model)  # Target network
    target_model.set_weights(model.get_weights())  # Synchronize weights

    replay_buffer = ReplayBuffer(buffer_size)

    for episode in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        for step in range(500):
            # Choose action using epsilon-greedy policy
            if np.random.rand() <= epsilon:
                action = np.random.choice(action_size)
            else:
                q_values = model.predict(state, verbose=0)
                action = np.argmax(q_values[0])

            # Perform the action
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            # Store experience in replay buffer
            replay_buffer.add((state, action, reward, next_state, done))

            # Update state
            state = next_state
            total_reward += reward

            # Training step
            if replay_buffer.size() >= batch_size:
                minibatch = replay_buffer.sample(batch_size)
                for s, a, r, s_next, d in minibatch:
                    target = model.predict(s, verbose=0)
                    if d:
                        target[0][a] = r
                    else:
                        t = target_model.predict(s_next, verbose=0)
                        target[0][a] = r + gamma * np.amax(t[0])

                    model.fit(s, target, verbose=0, batch_size=1)

            # Update target network periodically
            if step % update_target_freq == 0:
                target_model.set_weights(model.get_weights())

            if done:
                print(f"Episode {episode + 1}/{episodes}, Reward: {total_reward}, Epsilon: {epsilon:.2f}")
                break

        # Reduce exploration rate
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

    return model

# 5. Main execution
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    # Create or load a pre-trained base model
    base_model = create_base_model(input_dim=state_size, output_dim=2)  # Example pre-trained model

    # Train the agent in the new environment with transfer learning
    trained_model = train_dqn_with_transfer(env, base_model)
